In [ ]:
import json, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['axes.facecolor'] = 'white'
import matplotlib.colors

import pytz 

from datetime import timedelta, datetime

from research.utils.data_access_utils import RDSAccessUtils

In [ ]:
queryCache = {}

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

In [ ]:
dates = [
    '2020-10-01',
    '2020-10-02',
    '2020-10-03',
    '2020-10-04',
    '2020-10-05',
    '2020-10-06',
    '2020-10-07',
    '2020-10-08',
    '2020-10-09',
    '2020-10-10',
    '2020-10-11',
    '2020-10-12',
    '2020-10-13',
    '2020-10-14',
    '2020-10-15'
]

In [ ]:
def get_duration(annotation_state_id, start_date, end_date):
    query = """
        SELECT ca1.ts_source_start, ca1.ts_source_end FROM prod.crop_annotation ca1 
        INNER JOIN prod.crop_annotation ca2 on ca1.left_crop_url = ca2.left_crop_url
        WHERE ca1.service_id = 1 and ca1.annotation_state_id = 2 and ca1.captured_at >= '{1}' and ca1.captured_at <= '{2}' and ca1.left_crop_url is not null
            and ca2.service_id = 1 and ca2.annotation_state_id = {0} and ca2.captured_at >= '{1}' and ca2.captured_at <= '{2}' and ca2.left_crop_url is not null
        union
        SELECT ca1.ts_source_start, ca1.ts_source_end FROM prod.crop_annotation ca1 
        INNER JOIN prod.crop_annotation ca2 on ca1.right_crop_url = ca2.right_crop_url
        WHERE ca1.service_id = 1 and ca1.annotation_state_id = 2 and ca1.captured_at >= '{1}' and ca1.captured_at <= '{2}' and ca1.left_crop_url is null and ca1.right_crop_url is not null
            and ca2.service_id = 1 and ca2.annotation_state_id = {0} and ca2.captured_at >= '{1}' and ca2.captured_at <= '{2}' and ca2.left_crop_url is null and ca2.right_crop_url is not null
        ;
    """.format(annotation_state_id, start_date, end_date)

    if query in queryCache:
        return queryCache[query]
    
    df = rds_access_utils.extract_from_database(query)
    queryCache[query] = df
    
    return df

In [ ]:
def get_duration_qa(annotation_state_id, start_date, end_date):
    query = """
        SELECT ca1.ts_source_start, ca1.ts_source_end FROM prod.crop_annotation ca1 
        INNER JOIN prod.crop_annotation ca2 on ca1.left_crop_url = ca2.left_crop_url
        WHERE ca1.service_id = 1 and ca1.annotation_state_id = 5 and ca1.captured_at >= '{1}' and ca1.captured_at <= '{2}' and ca1.left_crop_url is not null
            and ca2.service_id = 1 and ca2.annotation_state_id = {0} and ca2.captured_at >= '{1}' and ca2.captured_at <= '{2}' and ca2.left_crop_url is not null
        union
        SELECT ca1.ts_source_start, ca1.ts_source_end FROM prod.crop_annotation ca1 
        INNER JOIN prod.crop_annotation ca2 on ca1.right_crop_url = ca2.right_crop_url
        WHERE ca1.service_id = 1 and ca1.annotation_state_id = 5 and ca1.captured_at >= '{1}' and ca1.captured_at <= '{2}' and ca1.left_crop_url is null and ca1.right_crop_url is not null
            and ca2.service_id = 1 and ca2.annotation_state_id = {0} and ca2.captured_at >= '{1}' and ca2.captured_at <= '{2}' and ca2.left_crop_url is null and ca2.right_crop_url is not null
        ;
    """.format(annotation_state_id, start_date, end_date)

    if query in queryCache:
        return queryCache[query]
    
    df = rds_access_utils.extract_from_database(query)
    queryCache[query] = df
    
    return df

In [ ]:
results = []

for index, date in enumerate(dates):
    if index == 0:
        continue

    start_date = dates[index - 1]
    end_date = date
    
    print(date)

    rejects = get_duration(4, start_date, end_date)
    #rejects = get_duration_qa(6, start_date, end_date)
    duration_rejects = rejects['ts_source_end'] - rejects['ts_source_start']
    secs_rejects = [d.total_seconds() for d in duration_rejects ]
    
    accepts = get_duration(3, start_date, end_date)
    #accepts = get_duration_qa(7, start_date, end_date)
    duration_accepts = accepts['ts_source_end'] - accepts['ts_source_start']
    secs_accepts = [d.total_seconds() for d in duration_accepts ]
    
    rt = np.median(secs_rejects) * len(secs_rejects)
    at = np.median(secs_accepts) * len(secs_accepts)
    tt = rt + at
    
    results.append((date, tt, rt, at, np.mean(secs_rejects), np.median(secs_rejects), len(secs_rejects), np.mean(secs_accepts), np.median(secs_accepts), len(secs_accepts)))

In [ ]:
for result in results:
    print(result[0], result[1] / 60 / 60 / 26)

In [ ]:
382112.0 / 60 / 60 / 26

In [ ]:
# 48 hours
results